# Case on Students Table
- Views
- SubQueries
- Window Functions
- Join
- Sorting, Filtering

# Views represent common classroom analytics:
	•	subject_avg_scores: how each subject is performing overall
	•	student_totals: personal performance summaries
	•	math_scores: focused subset of data
# Subqueries answer practical questions:
	•	Who’s above average overall?
	•	Which students have exceptionally high totals?
	•	What’s the average in my subject compared to my score?
	•	Who has beaten their subject’s average?
	•	What’s my latest recorded exam score?

# Check if you are using correct database
- mysql -u piuser -p
- Pass#123
- USE piit;
- DROP TABLE IF EXISTS student_scores

## Create Table
- CREATE TABLE student_scores ( id   INT PRIMARY KEY,  sname  VARCHAR(100) NOT NULL,  subject   VARCHAR(50)  NOT NULL,  examdate  DATE  NOT NULL,  score  DECIMAL(5,2) NOT NULL )

## Insert Data in the Table
- INSERT INTO student_scores (id, sname, subject, examdate, score) VALUES 
(1, 'Mandal', 'Math',    '2025-08-01', 85), 
(2, 'Amandeep', 'Math',    '2025-08-02', 75),
(3, 'Barkat', 'Math',      '2025-08-03', 92), 
(4, 'Suroor', 'Math',    '2025-08-04', 88), 
(5, 'Mandal',  'Science',    '2025-08-05', 70), 
(6, 'Amandeep', 'Science', '2025-08-06', 88), 
(7, 'Sharan',  'Math',    '2025-08-02', 75), 
(8, 'Barkat', 'Science', '2025-08-04', 92)

## Check the data
- SELECT * FROM student_scores;

## Views
### V1 : Avg score per subject
- CREATE OR REPLACE VIEW subject_avg_scores AS SELECT subject, AVG(score) AS avg_score, COUNT(*)  AS num_exams FROM student_scores GROUP BY subject;
- SELECT * FROM subject_avg_scores;
### V2 : Student total and average score
- CREATE OR REPLACE VIEW student_totals AS SELECT sname,  SUM(score) AS total_score, AVG(score) AS avg_score, COUNT(*)   AS exams_taken FROM student_scores GROUP BY sname;
- SELECT * FROM student_totals
- SELECT * FROM student_totals ORDER BY avg_score DESC
### V3 : View of only Math exam scores (updatable)
- CREATE OR REPLACE VIEW math_scores AS SELECT id, sname, subject, examdate, score FROM student_scores
WHERE subject = 'Math'
- SELECT * FROM math_scores 
- x
### V4 : 
- x
- x

## SUB-QUERIES
### SQ1 - Students whose total score is above the overall average score
- SELECT sname FROM student_scores GROUP BY sname HAVING SUM(score) > ( SELECT AVG(score) FROM student_scores)
- SELECT sname, SUM(score) FROM student_scores GROUP BY sname HAVING SUM(score) > ( SELECT AVG(score) FROM student_scores)
### SQ2 - Students whose total score is above the overall average score
- SELECT sname FROM student_scores GROUP BY sname HAVING SUM(score) > ( SELECT AVG(score) FROM student_scores)
### SQ3 - Inline view: students with total score > 250
- SELECT * FROM ( SELECT sname, SUM(score) AS total_score FROM student_scores    GROUP BY sname) AS totals WHERE total_score > 250
### SQ4 - Subquery in SELECT: each exam with subject average
- SELECT s1.id,  s1.sname,  s1.subject,  s1.score, (  SELECT AVG(s2.score)FROM student_scores AS s2  WHERE s2.subject = s1.subject) AS avg_in_subject FROM student_scores AS s1 ORDER BY subject, score DESC
### SQ5 -  EXISTS: students who have at least one score above their subject's average
- SELECT DISTINCT s1.sname FROM student_scores AS s1 WHERE EXISTS (     SELECT 1    FROM student_scores AS s2    WHERE s2.subject = s1.subject    GROUP BY s2.subject HAVING MAX(s1.score) > AVG(s2.score) );
### SQ6 - Latest exam for each student
- SELECT * FROM student_scores AS s1 WHERE s1.examdate = ( SELECT MAX(s2.examdate) FROM student_scores AS s2 WHERE s2.sname = s1.sname ) ORDER BY sname, examdate;  

#####

## WINDOWS functions
###  W1 - Rank within each subject by score (ties at same rank)
- SELECT   subject,  sname,  score,  RANK()       OVER (PARTITION BY subject ORDER BY score DESC)       AS subj_rank,  DENSE_RANK() OVER (PARTITION BY subject ORDER BY score DESC)       AS subj_dense_rank FROM student_scores ORDER BY subject, score DESC, sname;
### W2  - Overall ranking across all subjects (for a leaderboard view)
- SELECT   sname,  subject,  score,  RANK() OVER (ORDER BY score DESC) AS overall_rank,  DENSE_RANK() OVER (ORDER BY score DESC) AS overall_dense_rank
FROM student_scores ORDER BY score DESC, sname;
### W3 =  LAG/LEAD per student (chronological progression)
- Shows previous and next scores and the deltas (needs multiple rows per student; see optional inserts)
- SELECT   sname,  examdate,  subject,  score,
  LAG(score, 1)  OVER (PARTITION BY sname ORDER BY examdate)                AS prev_score,
  score - LAG(score, 1)  OVER (PARTITION BY sname ORDER BY examdate)        AS change_from_prev,
  LEAD(score, 1) OVER (PARTITION BY sname ORDER BY examdate)                AS next_score,
  LEAD(score, 1) OVER (PARTITION BY sname ORDER BY examdate) - score        AS change_to_next
FROM student_scores ORDER BY sname, examdate;
###  W4 - LAG within subject by score (who’s the next lower score in the same subject?)
- SELECT   subject,  sname,  score,  LAG(score) OVER (PARTITION BY subject ORDER BY score DESC) AS prev_higher_score,  score - LAG(score) OVER (PARTITION BY subject ORDER BY score DESC) AS gap_from_prev FROM student_scoresORDER BY subject, score DESC, sname

### W5 - A tidy leaderboard combining subject rank + overall rank
- WITH ranked AS (   SELECT    id, sname, subject, examdate, score,  RANK() OVER (PARTITION BY subject ORDER BY score DESC) AS subj_rank,  RANK() OVER (ORDER BY score DESC)   AS overall_rank FROM student_scores ) SELECT * FROM ranked ORDER BY subject, subj_rank, sname, examdate